# Preprocessing

## Loading our data from S3

In [ ]:
from pyspark.sql import functions as F

In [ ]:
filepath = "s3://full-stack-bigdata-datasets/Big_Data/youtube_playlog.csv"

In [ ]:
playlog = (spark.read.format('csv')\
             .option('header', 'true')\
             .option('inferSchema', 'true')\
             .load(filepath))
playlog.show(5)

+----------+----+-----------+
 timestamp|user| song|
+----------+----+-----------+
1392387533| 0|t1l8Z6gLPzo|
1392387538| 1|t1l8Z6gLPzo|
1392387556| 2|t1l8Z6gLPzo|
1392387561| 3|we5gzZq5Avg|
1392387566| 4|we5gzZq5Avg|
+----------+----+-----------+
only showing top 5 rows

## First analysis
1. Print out our DataFrame's schema

root
-- timestamp: integer (nullable = true)
-- user: integer (nullable = true)
-- song: string (nullable = true)

2. Use `.describe(...)` on your DataFrame

Out[7]:

,summary,timestamp,user,song
0,count,25739537,25739537,25739537
1,mean,1.4427006561045842E9,12697.352275450798,2.532571778181818E8
2,stddev,3.443284872371195E7,13094.065905828476,8.334645614940468E8
3,min,-139955897,0,---AtpxbkaE
4,max,1554321113,45903,zzzcFgRMY6c


### Missing values check

3. Count the missing values for each column put the result in a pandas DataFrame and print it out.
*TIP: you may use dictionnary comprehension in order to create the base to build the DataFrame from*


Out[8]:

,timestamp,user,song
missing values,0,0,0


### Duplicates check

4. Check if playlog without duplicates has the same number of rows as the original.

Out[9]: False

Seems like we have duplicates, let's count how many.

5. Figure out a way to count the number of duplicates.

Out[10]: 123651

### Other checks
6. Order the dataframe by ascending `timestamp` and show the first 5 rows.

+----------+----+-----------+
 timestamp|user| song|
+----------+----+-----------+
-139955897| 4|nRa-eGzpT6o|
1392387533| 0|t1l8Z6gLPzo|
1392387537| 70|VJ6ofd0pB_c|
1392387537| 22|Q24VZL8wpOM|
1392387538| 1|t1l8Z6gLPzo|
+----------+----+-----------+
only showing top 5 rows

Do you see anything suspicious?

The first timestamp is negative, and it seems like it's the only one.  
We will make sure there aren't other like this.

7. count the number of rows with a negative timestamp

Out[15]: 1

As expected, only one such negative timestamp. Since we have only one we can actually `.collect(...)` it.

8. Collect the problematic rows

Out[16]: [Row(timestamp=-139955897, user=4, song='nRa-eGzpT6o')]

There's only one problematic value among more than 25M.  This negative timestamp is an error, as such the real value is missing. We could try to reconstruct the real value but that would be a really tedious task, since it's one value over 25M, we will simply remove it.

## Removing the row with a negative timestamp

We will use our new knowledge about the data to perform some preprocessing.  

Our pipeline will have 2 steps:
* Remove duplicates (123651 rows)
* Remove row with negative timestamps (1 row)

We will call our new DataFrame `playlog_processed` and save it to S3 in parquet format.

9. Filter out:
* duplicated values
* rows with negative timestamp
* save the result to a new DataFrame: `playlog_processed`
* Finally, print out the number of rows in this DataFrame

Out[17]: 25615885

10. save the processed DataFrame to S3 using the parquet format for this you may use the the method .write.parquet(...)
*You may use this path 's3://full-stack-bigdata-datasets/Big_Data/playlog_processed_student.parquet'*